# 超小型红外收发器主板设计
## 尺寸目标：10×10mm

## 核心零件清单

In [ ]:
import pandas as pd

components = {
    "类别": ["MCU", "红外发射", "红外接收", "电源管理", "被动元件", "接口", "PCB"],
    "零件": ["STM32G030F6P6", "TSAL7600 + 100Ω电阻", "TSOP38438 + 1μF电容", 
            "MCP1700 + 4.7μF电容", "104电容×2 + 10k电阻", "半孔焊盘", "4层板"],
    "型号": ["TSSOP-20", "0805 + 0402", "TSOP-5 + 0402", 
            "SOT-23-3 + 0402", "0402×3", "0.5mm半孔", "ENIG 0.8mm"],
    "尺寸(mm)": ["3×3", "2.0×1.25", "2.7×2.4", "2.9×1.6", "1.0×0.5×3", "板边", "10×10×0.8"],
    "功能": ["主控制器", "940nm发射", "38kHz接收", "3.3V稳压", "退耦/复位", "板间连接", "电路载体"]
}

df = pd.DataFrame(components)
df.style.set_properties(**{'text-align': 'left'})

## 电路设计计算

In [ ]:
def calculate_ir_resistor(vcc, vf, desired_current):
    """计算红外发射管限流电阻值"""
    resistance = (vcc - vf) / desired_current
    return resistance

# 参数设置
VCC = 3.3    # 供电电压(V)
VF = 1.2     # IR LED正向压降(V)
CURRENT = 0.02  # 期望电流(A)

# 计算电阻值
r_value = calculate_ir_resistor(VCC, VF, CURRENT)
print(f"限流电阻计算值: {r_value:.0f} Ω")
print(f"实际选用: 100 Ω 0402封装 (1%精度)")

# 功率验证
power = r_value * (CURRENT ** 2)
print(f"\n电阻功率: {power*1000:.2f} mW (0402封装支持62.5mW，符合要求)")

## PCB布局规划
```
┌─────────────────┐
│  [IR RX]        │  <- TSOP38438 (贴顶部边缘)
│  ┌───┐          │
│  │MCU│ [IR LED] │  <- MCU旁紧贴发射管
│  └───┘          │
│  [LDO]  [CAP]   │  <- 电源模块角落布局
└─────────────────┘
尺寸：10mm×10mm（4层板）
```

## 制造规范

In [ ]:
from IPython.display import HTML

spec_table = """
<table>
  <tr><th>参数</th><th>要求</th><th>说明</th></tr>
  <tr><td>PCB层数</td><td>4层</td><td>信号-地-电源-信号</td></tr>
  <tr><td>板厚</td><td>0.8mm</td><td>±0.1mm公差</td></tr>
  <tr><td>铜厚</td><td>1oz (35μm)</td><td>内层/外层</td></tr>
  <tr><td>表面处理</td><td>ENIG</td><td>沉金，厚度≥0.05μm</td></tr>
  <tr><td>最小线宽</td><td>0.1mm</td><td>电源线≥0.3mm</td></tr>
  <tr><td>最小间距</td><td>0.1mm</td><td>信号线间</td></tr>
  <tr><td>过孔尺寸</td><td>0.2mm/0.1mm</td><td>激光钻孔</td></tr>
  <tr><td>半孔工艺</td><td>板边0.5mm半孔</td><td>树脂塞孔</td></tr>
  <tr><td>钢网厚度</td><td>0.08mm</td><td>激光切割</td></tr>
</table>
"""

HTML(spec_table)

## 测试验证点
1. **发射端**：IR LED阴极 - 检查38kHz PWM波形
2. **接收端**：MCU GPIO输入 - 验证信号完整性
3. **电源**：LDO输出 - 确认3.3V±2%
4. **功耗**：
   - 待机：<50μA
   - 发射峰值：≈30mA

## 备选方案对比

In [ ]:
import matplotlib.pyplot as plt

# 方案数据
solutions = {
    '方案': ['基础方案', '极限压缩', '商业模块'],
    '尺寸(mm²)': [100, 64, 35],
    '成本(USD)': [1.2, 1.8, 3.0],
    '灵活性': [5, 4, 2]  # 评分1-5
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 尺寸-成本散点图
ax1.scatter(solutions['尺寸(mm²)'], solutions['成本(USD)'], s=100)
for i, sol in enumerate(solutions['方案']):
    ax1.annotate(sol, (solutions['尺寸(mm²)'][i], solutions['成本(USD)'][i]+0.1))
ax1.set_title('尺寸 vs 成本')
ax1.set_xlabel('PCB面积 (mm²)')
ax1.set_ylabel('成本 (USD)')
ax1.grid(True)

# 灵活性雷达图
categories = ['尺寸', '成本', '灵活性', '易生产', '性能']
values_base = [4, 5, 5, 4, 4]  # 基础方案
values_mini = [5, 3, 4, 3, 4]   # 极限压缩
values_mod = [5, 2, 2, 5, 4]    # 商业模块

angles = [n / float(len(categories)) * 2 * 3.14159 for n in range(len(categories))]
angles += angles[:1]  # 闭合

ax2 = plt.subplot(122, polar=True)
ax2.plot(angles, values_base + values_base[:1], 'o-', label='基础方案')
ax2.plot(angles, values_mini + values_mini[:1], 'o-', label='极限压缩')
ax2.plot(angles, values_mod + values_mod[:1], 'o-', label='商业模块')
ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(categories)
ax2.set_title('方案对比雷达图')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()